This is the code implementation of the work for my MSc dissertation preparing a Racism detection model for tweets exploring avenues to reduce bias and demographic contexts. Please Note that only functional code snippets are provided rather than actual working implementation and duplication of steps are not presented.

In [ ]:
#used in pre-processing
import pandas as pd
import numpy as np
import re
import nltk
import sys
!pip install dataprep
from dataprep.clean import clean_country
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
import string
!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.MENTION,p.OPT.HASHTAG,p.OPT.RESERVED,p.OPT.NUMBER)
!pip install langdetect
from langdetect import detect
!pip install ekphrasis
from ekphrasis.classes.segmenter import Segmenter
seg = Segmenter(corpus="twitter") 
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
 
#used in EDA
!pip install chart_studio
!pip install textstat
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt

#used in model building
!pip install tensorflow_text
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.python.keras import regularizers
!pip install -q tf-models-official
from official.nlp import optimization
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.layers import Activation
from keras.layers import concatenate
from keras.models import Sequential
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.utils import class_weight
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
sys.setrecursionlimit(22000)

'''
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
pd.set_option('display.width', 300)
pd.set_option('display.max_colwidth', -1)
'''

Altering File 1 from Kaggle: https://kaggle.com/raghadabdullah/racism-tweets

In [ ]:
#upload files
from google.colab import files 
uploaded = files.upload()

In [ ]:
#reading to a df
tweets=pd.read_csv('tweets_PB1.csv')

In [ ]:
#mounting google drive with colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#loading trained-model from the drive
model_text = tf.keras.models.load_model('/content/drive/MyDrive/bert_cnn_bilst_low')

In [ ]:
#dropping and renaming columns
formatted1_df = unformatted_df.drop(columns=['index','oh_label','id'])
formatted1_df=formatted1_df.rename(columns={'Text':'text','Annotation':'annotation'})

Altering File 2 from Kaggle: https://kaggle.com/munkialbright/classified-tweets

In [ ]:
#dropping and renaming columns
formatted2_df = unformatted_df1.drop(columns=['suspicious','hate','suicidal'])
formatted2_df=formatted2_df.rename(columns={'cyberbullying':'annotation'})

#selecting racism/none categories
formatted2_df=formatted2_df.loc[formatted2_df['annotation'].isin(['0','1'])]

#relabelling 0:none, 1:racism
formatted2_df["annotation"]=formatted2_df["annotation"].replace({0:'none',1:'racism'})

In [ ]:
#merging both dfs and droppping index
final = pd.merge(formatted1_df, formatted2_df, on='text')
final=final.reset_index(drop=True)

In [ ]:
#removing duplicate texts
formatted2_df = formatted2_df.drop_duplicates('text')

In [ ]:
#saving as csv
final.to_csv('racism_P.csv', encoding='utf-8', index=False)

In [ ]:
#read tsv files
tweets=pd.read_csv('waseem_tweets.csv',delimiter=',')

Altering annotations and downloaded tweets: https://github.com/ZeerakW/hatespeech

In [ ]:
#creating a label from four annotators
labels['annotation'] = labels['Expert'].str.contains('racism') | labels['Amateur_0'].str.contains('racism') & labels['Amateur_1'].str.contains('racism') & labels['Amateur_2'].str.contains('racism')
labels['annotation'] = labels['annotation'].map({True: 'racism', False: 'none'})

#stripping spaces
tweets.columns = tweets.columns.str.strip()

Altering tweet file downloaded using Hydrator (Twitter API)

In [ ]:
#dropping unneded columns
tweets = tweets.drop(columns=['coordinates','created_at','hashtags','media','urls','favorite_count','in_reply_to_screen_name','in_reply_to_status_id','in_reply_to_user_id','lang','place','possibly_sensitive','retweet_count','retweet_id','retweet_screen_name','source','tweet_url','user_created_at','user_default_profile_image','user_favourites_count','user_listed_count','user_screen_name.1','user_statuses_count','user_time_zone','user_urls','user_verified'])

In [ ]:
#merging tweet and labels
waseemhovey_final=pd.merge(tweets, labels, on='id', how='outer')

In [ ]:
#appending both merged waseem & hovy datasets
wh_final = waseem_final.append(waseemhovey_final)

In [ ]:
#dropping tweets with null texts
tweets=wh_final.dropna(subset = ["text"])
#removing duplicate texts
tweets = tweets.drop_duplicates('text')

Pre-processing dataset

In [ ]:
#filtering non-english and extremely undecipherable tweets

#creates a new column with language name
def detect_lang(txt):
  try:
    return detect(txt)
  except:
    return np.nan
tweetstmp["language"] = tweetstmp.text.apply(detect_lang)

#selecting english tweets
tweetstmp = tweetstmp[tweetstmp.language == "en"]
tweetstmp=tweetstmp.reset_index(drop = True)

#dropping extra columns
tweetstmp=tweetstmp.drop(columns=['language'])

In [ ]:
#tweet processor to apply tweet cleaning
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text
tweetstmp['text'] = tweetstmp.apply(preprocess_tweet, axis=1)

In [ ]:
#removing special characters, numbers, whitespaces etc
tweetstmp['text'] = tweetstmp['text'].str.replace('[^\w\s]','').str.replace('\s\s+', ' ').str.replace('\[!.*?\]', '').str.replace('\n', '').str.replace('\w*\d\w*', '').str.replace('\d+', '')

In [ ]:
#word segmentation
tweetstmp['text']=[seg.segment(tweetstmp.loc[i,'text']) for i in range(len(tweetstmp))]
tweetstmp['user_description']=[seg.segment(tweetstmp.loc[i,'user_description']) for i in range(len(tweetstmp))]
tweetstmp['user_name']=[seg.segment(tweetstmp.loc[i,'user_name']) for i in range(len(tweetstmp))]
tweetstmp['user_screen_name']=[seg.segment(tweetstmp.loc[i,'user_screen_name']) for i in range(len(tweetstmp))]

In [ ]:
#extracting standardised locations

#returns new column with location
tweetstmp=clean_country(tweetstmp, 'user_location')

#dropping old field and renaming new one
tweetstmp = tweetstmp.drop(columns=['user_location'])
tweetstmp=tweetstmp.rename(columns={'user_location_clean':'user_location'})

#lowercasing
tweetstmp['user_location']=tweetstmp['user_location'].str.lower()

In [ ]:
#sentiment analysis
analyzer = SentimentIntensityAnalyzer()

#new column with compound scores
tweetstmp['compound'] = [analyzer.polarity_scores(x)['compound'] for x in tweetstmp['text']]

#labelling score values to sentiment labels
conditions = [
    (tweetstmp['compound'] >= 0.05),
    (tweetstmp['compound'] <= - 0.05)
    ]
values = ['Positive', 'Negative']
tweetstmp['sentiment'] = np.select(conditions, values)

tweetstmp['sentiment']=tweetstmp['sentiment'].replace('0','neutral')

#lowercasing & dropping extra column
tweetstmp['sentiment']=tweetstmp['sentiment'].str.lower()
tweetstmp = tweetstmp.drop(columns=['compound'])

In [ ]:
#dropping non-essential text features for reducing skew of the model due to low size and imbalanced dataset
tweetstmp = tweetstmp.drop(columns=['user_screen_name','user_description','user_name'])

In [ ]:
#binarizing labels
tweetstmp['annotation']=np.where(tweetstmp['annotation']=='racism', 1, 0)

In [ ]:
#handling nans
tweetstmp.fillna(value='', inplace=True)

In [ ]:
#one hot enocding
one_hot = pd.get_dummies(tweets['sentiment'])
tweets = tweets.drop('sentiment',axis = 1)
tweets = tweets.join(one_hot)

In [ ]:
#normalization using quantile transformer
qt = QuantileTransformer()
tweets_nn = pd.DataFrame(qt.fit_transform(tweets_nn),columns = tweets_nn.columns)

EDA

In [ ]:
#displaying a sample of each label
print("Racist Tweet example :",tweets[tweets['annotation']==1]['text'].values[8])
print("Non-Racist Tweet example :",tweets[tweets['annotation']==0]['text'].values[0])

In [ ]:
#count of the features
tweets.count()

In [ ]:
#distribution of the dataset
tweets['annotation'].value_counts()
tweets['annotation'].value_counts(normalize=True)

In [ ]:
#distribution of sentiment features for each label
tweets.groupby(["annotation", "sentiment"]).size()

In [ ]:
#distribution of locations per label
tweets.groupby(["annotation", "user_location"]).size()

In [ ]:
#creating separate dfs for each labels
racist = tweets[tweets['annotation']==1]
none = tweets[tweets['annotation']==0]

In [ ]:
#function to return top words
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

#function to return top n-grams
def get_top_n_gram(corpus,ngram_range,n=None):
    vec = CountVectorizer(ngram_range=ngram_range,stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
#top 10 words in racist tweets
rac_unigrams = get_top_n_words(racist['text'],10)

#top 10 words in normal tweets
non_unigrams = get_top_n_words(none['text'],10)

#top 10 racist bigrams
rac_bigrams = get_top_n_gram(racist['text'],(2,2),10)

#top 10 normal bigrams
non_bigrams = get_top_n_gram(none['text'],(2,2),10)

#top 10 racist trigrams
rac_trigrams = get_top_n_gram(racist['text'],(3,3),10)

#top 10 normal trigrams
non_trigrams = get_top_n_gram(none['text'],(3,3),10)

In [ ]:
#word cloud
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[20, 10])
wordcloud1 = WordCloud( background_color='white',
                        width=300,
                        height=250).generate(" ".join(none['text']))
ax1.imshow(wordcloud1)
ax1.axis('off')
ax1.set_title('Normal Tweets',fontsize=30);

wordcloud2 = WordCloud( background_color='white',
                        width=300,
                        height=250).generate(" ".join(racist['text']))
ax2.imshow(wordcloud2)
ax2.axis('off')
ax2.set_title('Racist Tweets',fontsize=30);

Model Building

In [ ]:
#downloading and loading Bert pre-processor and encoder from Tensorflow
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4")

In [ ]:
#function to test the encoder. returns word embeddings as tensors
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']    

In [ ]:
#splitting data for test and train
x_train, x_test, y_train, y_test = train_test_split(tweets['text'],tweets['annotation'],stratify=tweets['annotation'], test_size=0.2)

Text Model-BERT+CNN+bi-LSTM

In [ ]:
#functional model

#bert word embeddings
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_inputs')
preprocessed_text = bert_preprocess(text_input)
text_outputs = bert_encoder(preprocessed_text)
out=(text_outputs['pooled_output'])

#reshaping 2D tensor to 3D for the model
out1=tf.keras.layers.Reshape([1024,1], name='reshape1')(out)

#cnn layers
cnn1 = keras.layers.Conv1D(filters=64,kernel_size=2, activation='relu', padding='same', kernel_regularizer=keras.regularizers.l2(0.1), name='cnn1')(out1)
cnn2 = keras.layers.Conv1D(filters=64,kernel_size=3, activation='relu', padding='same', kernel_regularizer=keras.regularizers.l2(0.1), name='cnn2')(out1)
cnn3 = keras.layers.Conv1D(filters=64,kernel_size=4, activation='relu', padding='same', kernel_regularizer=keras.regularizers.l2(0.1), name='cnn3')(out1)
cnn4 = keras.layers.Conv1D(filters=64,kernel_size=5, activation='relu', padding='same', kernel_regularizer=keras.regularizers.l2(0.1), name='cnn4')(out1)

#maxpooling layers
pool1=tf.keras.layers.MaxPooling1D(pool_size=2,name='pool1')(cnn1)
pool2=tf.keras.layers.MaxPooling1D(pool_size=2,name='pool2')(cnn2)
pool3=tf.keras.layers.MaxPooling1D(pool_size=2,name='pool3')(cnn3)
pool4=tf.keras.layers.MaxPooling1D(pool_size=2,name='pool4')(cnn4)

#dropout layers
d1 = tf.keras.layers.Dropout(rate=0.1, name="dropout1")(pool1)
d2 = tf.keras.layers.Dropout(rate=0.1, name="dropout2")(pool2)
d3 = tf.keras.layers.Dropout(rate=0.1, name="dropout3")(pool3)
d4 = tf.keras.layers.Dropout(rate=0.1, name="dropout4")(pool4)

#concatenating cnn outputs
concat=tf.keras.layers.concatenate([d1,d2,d3,d4], name='concat')

#bi-lstm layer
bi1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=32),  name='bi-lstm1')(concat)

#dropout layer
d5 = tf.keras.layers.Dropout(rate=0.1, name="dropout5")(bi1)

#classification dense layer
dns1 = tf.keras.layers.Dense(units=1, activation='sigmoid', name="bert_outputs")(d5)

#model definition
model_text = tf.keras.Model(inputs=[text_input], outputs = [dns1])

In [ ]:
#summary of the model
model_text.summary()

In [ ]:
#defining optimizer and learning rate
opt = tf.keras.optimizers.Adam(5e-5)

#defining an early stoping function to control epochs based on model loss
es = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

#compiling model
model_text.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_accuracy'])

#defining class weights to counter label distibution imbalance
class_weights = dict(zip(np.unique(y_train), class_weight.compute_class_weight('balanced', np.unique(y_train), y_train))) 

#training the model
bert_cnn_bilstm=model_text.fit(x_train,y_train, epochs=100, batch_size=32, class_weight=class_weights, verbose=1, callbacks=[es])

In [ ]:
#reset learned weights
model_text.reset_states()

In [ ]:
#saving trained model
model_text.tf.keras.models.save('bert_cnn_bilstm_low')

#loading saved model
model_text = tf.keras.models.load_model('bert_cnn_bilstm_low')

In [ ]:
#evaluating model on test set
model_text.evaluate(x_test,y_test)

In [ ]:
#sample real-world predictions
sample_text = (['the bengalis are hoarding the state and littering it',
                'hello darling',
                'deport your black ass back to Africa nigga',
                'tommorrow is a good day',
                'live and let live says a wise man',
                'people deserve to live',
                'asians are shit and should be kept in their offices',
                'Messis debut is something to look forward to',
                'The guys wont be back until these German twits leave the table'])
predictions = model_text.predict(sample_text)
predictions

In [ ]:
#function to return roc_auc score
def roc_auc(predictions,target):
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [ ]:
#running predictions and binarizing result
predictions = model_text.predict(x_test)
predictions=np.where(predictions>0.5,1,0)

#confusion matrix
results = confusion_matrix(y_test, predictions)
print(results)

#classification report
print(classification_report(y_test,predictions))

#roc_auc score
scores_model = []
scores_model.append({'Model': 'Bert_CNN_biLSTM','AUC_Score': round(roc_auc(model_text.predict(x_test),y_test),2)})
scores_model

Numerical Model-Dense Layers

In [ ]:
#sequential model
num_model = Sequential()
num_model.add(tf.keras.layers.Dense(128, activation='relu',name ='dense1'))
num_model.add(tf.keras.layers.Dropout(rate=0.1, name='drop1'))
num_model.add(tf.keras.layers.Dense(1, activation='sigmoid', name='dense2'))
num_model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])

Combined Model

In [ ]:
#splitting data for combined model(3 inputs)
x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(tweets['text'],tweets[tweets.columns.difference(['text', 'annotation'])], tweets['annotation'],stratify=tweets['annotation'], test_size=0.2)

In [ ]:
#combining trained model outputs
combined = concatenate([model_text.output, num_model.output])

#layers
fdense1 = tf.keras.layers.Dense(256, activation="relu", name ='fdense1')(combined)
fd1= tf.keras.layers.Dropout(rate=0.1, name='fdrop1')(fdense1)
fdense2 = tf.keras.layers.Dense(1, activation="sigmoid", name ='fdense2')(fd1)

#defining model with inputs of pre-built model combined with the final layers
combined_model = tf.keras.Model(inputs=[model_text.input, num_model.input], outputs=fdense2)

In [ ]:
#training combined model
final_nn=combined_model.fit([x_train, y_train], z_train, epochs=100, batch_size=32, class_weight=class_weights, verbose=1, callbacks=[es])

In [ ]:
#evaluating combined model
combined_model.evaluate([x_test,y_test],z_test)

In [ ]:
#running predictions and binarizing result
predictions = model_text.predict([x_test,y_test])
predictions=np.where(predictions>0.5,1,0)

#confusion matrix
results = confusion_matrix(z_test, predictions)
print(results)

#classification report
print(classification_report(z_test, predictions))

#roc_auc score
scores_model = []
scores_model.append({'Model': 'Dense','AUC_Score': round(roc_auc(combined_model.predict([x_test,y_test]),z_test),2)})
scores_model